In [1]:
!pip3 install findspark

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip3 install numpy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 53.7 MB/s eta 0:00:0000:0100:01


In [2]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')
findspark.find()

'/home/ubuntu/spark-3.3.1-bin-hadoop3'

In [3]:
from pyspark.sql import SparkSession

# The entry point into all functionality in Spark is the SparkSession class.
spark = (SparkSession
	.builder
	.appName("DS5110/CS5501: DDoS detection project")
	.master("spark://172.31.24.89:7077")
	.config("spark.executor.memory", "1024M")
	.getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/11 01:31:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Load csv data into dataframe

In [4]:
df = (spark.read
         .format("csv")
         .option("inferSchema", True)
         .option("header", True)
         .load("hdfs://172.31.24.89:9000/02-21-2018.csv"))

In [7]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

Data preprocessing

In [15]:
from pyspark.sql.functions import col, when
#drop unused columns
df = df.drop('Timestamp')
df = df.na.drop()
df = df.withColumn("Label", when(col("Label") == "Benign", 1).otherwise(0))

In [16]:
features = df.columns[:-1]
label = df.columns[-1]
# Replace missing values in 'Label' with the most common value (mode)
# mode = labels.groupBy("Label").count().orderBy("count", ascending=False).first()["Label"]
# labels = labels.na.fill(value=mode, subset=["Label"])

In [17]:
# Assemble the features into a single vector (necessary for Spark MLlib models)
assembler = VectorAssembler(inputCols=features, outputCol="features")
data = assembler.transform(df)
# Select the features vector and the label column
data = data.select("features", label)
# Split the data into training and test sets
trainData, testData = data.randomSplit([0.7, 0.3])

In [18]:
# Define and fit the Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(trainData)

In [19]:
predictions = lrModel.transform(testData)

In [20]:
# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="Label")
accuracy = evaluator.evaluate(predictions)

print(f"Accuracy: {accuracy}")

Accuracy: 0.9965734461193536
